# Cats and Dog Classification
In this data i have used functional api to predict if its a DOG or CAT.

### Importing Libraries and data from kaggle
I have deleted my kaggle key just for security purpose

In [1]:
import os
os.environ['KAGGLE_USERNAME']='maosama'
os.environ['KAGGLE_KEY']=''

In [2]:
# The work was done on google collab so i directly download the data and unzipped it
# i have hide the unzip process cause it is too big.
! kaggle datasets download -d shaunthesheep/microsoft-catsvsdogs-dataset

100% 785M/788M [00:07<00:00, 96.4MB/s]
100% 788M/788M [00:07<00:00, 109MB/s] 


In [ ]:
! unzip /content/microsoft-catsvsdogs-dataset.zip

## Data Preprocessing

In [4]:
data_dir = "/content/PetImages/"
os.listdir(data_dir)

['Cat', 'Dog']

In [5]:
import cv2
data_dir = "/content/PetImages"
labels = ["Cat", "Dog"]
x = []
y = []
for label in labels:
    data = os.path.join(data_dir,label)
    for image in os.listdir(data):
        try:
            im = cv2.imread(os.path.join(data,image),cv2.IMREAD_COLOR)
            im = cv2.resize(im,(128,128))
            # Using the Canny filter with different parameters
            
            x.append(im)
            y.append(labels.index(label))
            
            
        except Exception as e:
            pass

In [6]:
import numpy as np
np.unique(y)
np.unique(x)

x = np.array(x)
x = x.reshape(-1, 128, 128, 3)

In [7]:
from sklearn.model_selection import train_test_split   
x_train, x_test, y_train, y_test = train_test_split(x, y)

In [8]:
num_labels = len(np.unique(y_train))

num_labels       

from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train) 
y_test = to_categorical(y_test)

image_size = x_train.shape[1]

x_train = np.reshape(x_train, [-1,image_size, image_size, 3]).astype("float32")/255
x_test = np.reshape(x_test, [-1,image_size, image_size, 3]).astype("float32")/255

input_shape = (image_size, image_size, 3)

In [9]:
# Network Parameter
batch_size = 32 
kernel_size = 7
dropout = 0.3
n_filters = 8

## Functional API model building

In [10]:
# Left branch of Y network
from tensorflow.keras.layers import Input, Conv2D, Flatten, Dense, concatenate, MaxPooling2D, Dropout
left_inputs = Input(shape=input_shape)
x1 = left_inputs
filters = n_filters

for i in range(3):
    x1 = Conv2D(filters, kernel_size, padding="same", activation="relu")(x1)
    x1 = Dropout(dropout)(x1)
    x1 = MaxPooling2D()(x1)
    filters *= 2

In [11]:
# Right Branch y
right_inputs = Input(shape = input_shape)
y1 = right_inputs

for i in range(3):
    y1 =  Conv2D(filters, kernel_size, padding="same", activation="relu", dilation_rate=2)(y1)
    y1 = Dropout(dropout)(y1)
    y1 = MaxPooling2D()(y1)
    filters *= 2

In [12]:
# Merge left and right branches output
y2 = concatenate([x1,y1])

# Feture map to vector before connecting to Dense
y2 = Flatten()(y2)
y2 = Dropout(dropout)(y2)

outputs = Dense(2, activation="sigmoid")(y2)

In [13]:
# Now turn the model in functional to train
from tensorflow.keras.models import Model

model = Model([left_inputs, right_inputs], outputs)
model.summary()

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 128, 128, 8)  1184        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_3 (Conv2D)               (None, 128, 128, 64) 9472        input_2[0][0]                    
______________________________________________________________________________________________

In [14]:
# Train the model
model.fit([x_train, x_train],y_train, validation_data=([x_test, x_test], y_test), epochs=20,
          batch_size = batch_size)

Epoch 1/20
585/585 [==============================] - 124s 154ms/step - loss: 0.6971 - accuracy: 0.5727 - val_loss: 0.6180 - val_accuracy: 0.6726
Epoch 2/20
585/585 [==============================] - 89s 152ms/step - loss: 0.5803 - accuracy: 0.6967 - val_loss: 0.5882 - val_accuracy: 0.6902
Epoch 3/20
585/585 [==============================] - 89s 152ms/step - loss: 0.5020 - accuracy: 0.7592 - val_loss: 0.5238 - val_accuracy: 0.7563
Epoch 4/20
585/585 [==============================] - 89s 151ms/step - loss: 0.4506 - accuracy: 0.7903 - val_loss: 0.5151 - val_accuracy: 0.7425
Epoch 5/20
585/585 [==============================] - 89s 151ms/step - loss: 0.4303 - accuracy: 0.7988 - val_loss: 0.4669 - val_accuracy: 0.7775
Epoch 6/20
585/585 [==============================] - 88s 151ms/step - loss: 0.3932 - accuracy: 0.8198 - val_loss: 0.4439 - val_accuracy: 0.8065
Epoch 7/20
585/585 [==============================] - 88s 151ms/step - loss: 0.3802 - accuracy: 0.8241 - val_loss: 0.4323 - val_a

## As you can see the accuracy and val accuracy is 89 and 81.
## I had the higher expectation but i guess data was not complex enough to be used in Functional api.